In [68]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np

In [39]:
scores['year_event_golfer'] = (scores['year'].astype(str) +
                                      '_' + scores['event_id'].astype(str) +
                                      '_' + scores['dg_id'].astype(str)
                              )

early_outs = ['CUT','DQ','WD', 'W/D', 'MDF',]
early_out_num = 75

scores['early_out'] = np.where(scores.fin_text.isin(early_outs),1,0)

scores['fin_num'] = (np.where(scores.fin_text.isin(early_outs),
                             early_out_num,
                             scores.fin_text.str.replace('T',''))
                    ).astype(int)

# Aggregating round data to an event level
event_df = (scores.groupby(['dg_id','player_name','event_name',
                            'event_id','fin_text','fin_num','early_out',
                            'year','year_event_golfer','event_completed',]).
            agg(rounds=('round','count'),
                mean_score = ('score', 'mean'),
                mean_sg = ('sg_total', 'mean')
               ).reset_index().sort_values('event_completed')
           )

In [53]:
rahm_fowler = event_df.query("dg_id in (19195,12965)").reset_index(drop=True)

In [65]:
def create_rolling_agg_features_by_golfer(df, field, n_shift=1, n_rolling, agg_func):
    """
    """
    df = df.copy()
    
    df['shifted_field'] = (df.groupby('dg_id')[field]
                           .shift(n_shift)
                           .fillna(0)
                          )
    
    return (df.groupby('dg_id')['shifted_field']
            .transform(lambda x: x.rolling(n_rolling, min_periods=1)
                       .agg(agg_func))
           )

fields = 'early_out'

    

In [75]:
metric_lst = [
    {'field': 'early_out','periods': [1, 2, 5], 'agg': 'sum'},
    {'field': 'fin_num', 'periods': [1, 2, 5], 'agg': 'mean'},
    {'field': 'mean_sg', 'periods': [1, 2, 5], 'agg':'mean'},
    ]


In [76]:
for metric in metric_lst:
    field = metric.get('field')
    agg = metric.get('agg')
    for period in metric.get('periods'):
        col = f"{field}_in_prev_{period}_events"
        event_df[col] = create_rolling_agg_features_by_golfer(event_df, field, 1, period, agg)

In [78]:
event_df.tail()

,dg_id,player_name,event_name,event_id,fin_text,fin_num,early_out,year,year_event_golfer,event_completed,rounds,mean_score,mean_sg,early_out_in_prev_1_events,early_out_in_prev_2_events,early_out_in_prev_5_events,fin_num_in_prev_1_events,fin_num_in_prev_2_events,fin_num_in_prev_5_events,mean_sg_in_prev_1_events,mean_sg_in_prev_2_events,mean_sg_in_prev_5_events
12527,12359,"Stallings, Scott",THE NORTHERN TRUST,27,CUT,75,1,2021,2021_27_12359,2021-08-22,2,72.5,-2.3495,1.0,1.0,1.0,75.0,46.5,42.4,-0.00300,0.485750,0.56310
23476,18580,"Gooch, Talor",THE NORTHERN TRUST,27,T31,31,0,2021,2021_27_18580,2021-08-22,4,69.0,0.6985,1.0,1.0,1.0,75.0,54.0,35.2,-0.50300,0.224625,0.78915
16316,14050,"Sloan, Roger",THE NORTHERN TRUST,27,CUT,75,1,2021,2021_27_14050,2021-08-22,2,75.5,-5.3495,0.0,0.0,0.0,2.0,4.0,25.2,2.56275,2.768625,1.31595
2239,5951,"Gay, Brian",THE NORTHERN TRUST,27,CUT,75,1,2021,2021_27_5951,2021-08-22,2,72.5,-2.3495,0.0,1.0,3.0,51.0,63.0,68.0,0.31275,0.170125,-0.49800
13407,12808,"Grillo, Emiliano",THE NORTHERN TRUST,27,CUT,75,1,2021,2021_27_12808,2021-08-22,2,71.0,-0.8495,0.0,1.0,3.0,27.0,51.0,52.8,0.72450,-0.642500,-0.08735
